### LCPs

In [ ]:
import torch
import torch.nn as nn   
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from math import pi, cos
from scipy.special import factorial
from sympy import symbols, lambdify
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
from sympy import simplify
from scipy.sparse import diags
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import LBFGS

In [ ]:
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")  # Check for CUDA availability

m = 2  # degree of polynomial Q(x)
n = 2  # degree of polynomial P(x)
q = 1  # degree of truncated Taylor series,q<=n
J = 16  # for loss_1
N = 10000  # for lambda values in loss_1
h = 1/N  # for lambda values in loss_1
p2 = 1  # penalty term for loss_2
epochs = 500
K = 500 # loops in one epoch

# coefficients of exp(-x) Taylor expansion
c = torch.tensor([(-1)**i / factorial(i) for i in range(q+1)], dtype=torch.float32, device=device)

# Padé approximation parameters to be learned
a2 = nn.Parameter(torch.zeros(n-q, device=device), requires_grad=True)
b1_log = nn.Parameter(torch.zeros(q, device=device), requires_grad=True)
b2_log = nn.Parameter(torch.ones(m-q, device=device), requires_grad=True)



In [ ]:
def P(x):
    """
    Polynomial function with coefficients a = [c*B, a2]
    """
    B = torch.cat((torch.ones(1, device=device), torch.exp(b1_log)))
    c_B = torch.zeros_like(B, device=device)
    for k in range(len(B)):
        for j in range(k+1):
            c_B[k] += c[j] * B[k-j]
    a = torch.cat((c_B, a2))
    p = torch.zeros_like(x, device=device)
    for coeff in a.flip(0):  # Reverse the tensor for Horner's method
        p = p * x + coeff
    return p

def Q(x):
    """
    Polynomial function with coefficients b = [B, b2]
    """
    B = torch.cat((torch.ones(1, device=device), torch.exp(b1_log)))
    b = torch.cat((B, torch.exp(b2_log)))
    q = torch.zeros_like(x, device=device)
    for coeff in b.flip(0):  # Reverse the tensor for Horner's method
        q = q * x + coeff
    return q

# LCPs
def R(x):
    return P(x) / Q(x)

# two-stage Lobatto IIIC 
def r2(x):
    return (2) / (x ** 2 + 2 * x + 2) 

# three-stage Lobatto IIIC
def r3(x):
    return (-6 * x + 24) / (x ** 3 + 6 * x ** 2 + 18 * x + 24)

# four-stage Lobatto IIIC
def r4(x):
    return (12*x**2 - 120*x + 360) / (x**4 + 12*x**3 + 72*x**2 + 240*x + 360)

# three-stage Radau IIA
def rc(s):
    b = 0.5 * (1 + np.sqrt(3) / 3)
    return 1 - s / (1 + b * s) - np.sqrt(3) / 6 * (s / (1 + b * s))**2

In [ ]:
def solve_lambda_zeros(b1, b2, a2):
    A = a2 * torch.exp(b1) + torch.exp(b2) - torch.exp(b1 + b2)
    B = -2 * (torch.exp(b2) - a2)
    C = -1
    
    discriminant = B**2 - 4 * A * C
    
    if torch.any(discriminant < 0):
        raise ValueError("The equation has no real roots for some inputs.")
    
    sqrt_discriminant = torch.sqrt(discriminant)
    
    root1 = (-B + sqrt_discriminant) / (2 * A)
    root2 = (-B - sqrt_discriminant) / (2 * A)
    
    return root1, root2


In [ ]:

lambdas = torch.tensor([-(2*cos(j*pi*h) - 2)/(h**2) for j in range(1, N)], dtype=torch.float32).to(device)
lambdas2 = torch.linspace(0.01, 100, 5000, device=device)

lambdas = torch.cat((lambdas, lambdas2))

# Define the dataset
dataset = torch.utils.data.TensorDataset(lambdas)

# Define the data loader with batch size equal to the total number of samples
data_loader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset))

# Define an optimizer with Adam
optimizer = torch.optim.SGD([a2, b1_log, b2_log], lr=1e-4)
# optimizer = torch.optim.Adam([a2, b1_log, b2_log], lr=0.01)

# Define a learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=K, eta_min=0)

best_params = None
best_loss1 = float('inf')

# 定义收敛阈值
convergence_threshold = 4e-1

recent_losses = []

for epoch in range(epochs):
    p2 = p2 * 0.95
    converged = False
    
    for i in range(K):
        optimizer.zero_grad()
        
        # 计算 loss1
        loss1 = torch.max(torch.abs((r4(lambdas/J))**J - R(lambdas))/(1 - torch.abs(R(lambdas))))
        # loss1 = torch.max(torch.abs((rc(lambdas/J))**J - R(lambdas))/(1 - torch.abs(R(lambdas))))
        # loss1 = torch.max(torch.abs((r3(lambdas/J))**J - R(lambdas))/(1 - torch.abs(R(lambdas))))
        # loss1 = torch.max(torch.abs((r2(lambdas/J))**J - R(lambdas))/(1 - torch.abs(R(lambdas))))
        # loss1 = torch.max(a2 + b1_log + b2_log)
        
        # 计算 loss2
        root1, root2 = solve_lambda_zeros(b1_log, b2_log, a2)
        pos_lambdas = torch.max(torch.max(root1, torch.tensor(0.001, device=device)), torch.max(root2, torch.tensor(0.001, device=device)))
        pos_lambdas = torch.cat((pos_lambdas, lambdas))
        loss2 = torch.mean(torch.log(1 - R(lambdas)**2) + torch.log(1 - (a2 / torch.exp(b2_log))**2))

        # 组合损失
        loss = loss1 - p2 * loss2
        loss.backward(retain_graph=True)
        optimizer.step()
        scheduler.step()

        # 如果 loss1 小于当前最小值，则更新 best_params
        if loss1 < best_loss1:
            best_params = (a2.clone().detach(), b1_log.clone().detach(), b2_log.clone().detach())
            best_loss1 = loss1.item()


        if i % 100 == 0:
            # 重新计算 loss1 的梯度
            optimizer.zero_grad()
            loss1.backward()
            
            # 计算梯度范数
            total_norm = 0
            for p in [a2, b1_log, b2_log]:
                if p.grad is not None:
                    param_norm = p.grad.data.norm(2)
                    total_norm += param_norm.item() ** 2
            total_norm = total_norm ** 0.5

            # 仅在 i=0 时跟踪 recent_losses
            if i == 0:
                recent_losses.append(loss1.item())
                if len(recent_losses) > 3:
                    recent_losses.pop(0)

            print(f"Iteration {epoch}, Iteration: {i}, Loss1: {loss1:.6f}, Loss2: {loss2:.6f},gradient norm {total_norm:.6f}, Total Loss: {loss.item():.6f}")
            
            # 检查梯度范数以确定收敛性
            if total_norm < convergence_threshold:
                print(f"Converged at epoch {epoch}, iteration {i}, gradient norm {total_norm:.6f}")
                converged = True
                break
            
        
    
    # 检查整个 epoch 是否收敛
    if converged:
        break

In [ ]:
a2.data = best_params[0]
b1_log.data = best_params[1]
b2_log.data = best_params[2]

B = torch.cat((torch.ones(1, device=device), torch.exp(b1_log)))
c_B = torch.zeros_like(B, device=device)
for k in range(len(B)):
    for j in range(k + 1):
        c_B[k] += c[j] * B[k - j]

a = torch.cat((c_B, a2))
b = torch.cat((torch.ones(1, device=device), torch.exp(b1_log), torch.exp(b2_log)))

Px = " + ".join([f"{a[i].item():.5f}*x**{i}" for i in range(len(a))])
Qx = " + ".join([f"{b[i].item():.5f}*x**{i}" for i in range(len(b))])

print(f"R_x = ({Px}) / ({Qx})\nThe best convergence rate is {best_loss1:.6f}")

In [ ]:
# Define symbolic variable
x = symbols('x')

# Define the function r(x), keeping 4 decimal places
R_x = (1.00000*x**0 + -0.21151*x**1 + 0.00500*x**2) / (1.00000*x**0 + 0.78849*x**1 + 0.37864*x**2)

# Define the p_i function, replacing r with r(x)
p1_x = -(R_x - 1)/x

# Simplify p_i
p1_simplified = simplify(p1_x)

# Output the simplified p_i
print("p_1(x) =", p1_simplified)
